In [249]:
import pandas as pd
import numpy as np
import re
import csv
import os

In [70]:
lst =os.listdir('C:/Users/PC/Desktop/Settlements - Copy')

In [60]:
lst

['10-Year T-Note_10_4_2019_int_settlements.txt',
 '10-Year T-Note_10_5_2017_int_settlements.txt',
 '10-Year T-Note_11_9_2017_int_settlements.txt',
 '10-Year T-Note_12_12_2018_int_settlements.txt',
 '10-Year T-Note_12_3_2019_int_settlements.txt',
 '10-Year T-Note_13_6_2017_int_settlements.txt',
 '10-Year T-Note_14_6_2017_int_settlements.txt',
 '10-Year T-Note_14_9_2017_int_settlements.txt',
 '10-Year T-Note_15_2_2018_int_settlements.txt',
 '10-Year T-Note_16_10_2017_int_settlements.txt',
 '10-Year T-Note_16_5_2017_int_settlements.txt',
 '10-Year T-Note_17_1_2019_int_settlements.txt',
 '10-Year T-Note_17_4_2017_int_settlements.txt',
 '10-Year T-Note_18_12_2018_int_settlements.txt',
 '10-Year T-Note_18_6_2019_int_settlements.txt',
 '10-Year T-Note_19_3_2019_int_settlements.txt',
 '10-Year T-Note_19_7_2017_int_settlements.txt',
 '10-Year T-Note_1_3_2019_int_settlements.txt',
 '10-Year T-Note_20_12_2018_int_settlements.txt',
 '10-Year T-Note_20_9_2018_int_settlements.txt',
 '10-Year T-Note_

In [71]:
archs = pd.DataFrame([l.split('_') for l in lst])

In [72]:
list(set(archs[0])) 

['soybean oil',
 'SnP',
 'soybeans',
 '20',
 'S&P',
 '1',
 '11',
 '16',
 'Crude Oil',
 'kansas city wheat',
 '6',
 '8',
 'wheat',
 'Soybean Oil',
 '18',
 '30',
 '27',
 '30-Year T-Note',
 '19',
 '10',
 '5',
 'Soybean Meal',
 '22',
 '25',
 'Soybeans',
 '24',
 '9',
 'soybean meal',
 '26',
 'Kansas City Wheat',
 'Wheat',
 '29',
 '4',
 'corn',
 '3',
 '21',
 '23',
 '17',
 '10-Year T-Note',
 '2',
 '13',
 '14',
 '12',
 '15',
 '31',
 'Corn',
 '7',
 '28']

In [85]:
archs_corn = archs[ (archs[0]=='Corn') | (archs[0]=='corn') ]
archs_wheat = archs[ (archs[0]=='Wheat') | (archs[0]=='wheat') ]
archs_soybean = archs[ (archs[0]=='Soybeans') | (archs[0]=='soybeans') ]

In [89]:
arches_corn = archs_corn.sort_values(by=[3,2,1])
arches_wheat = archs_wheat.sort_values(by=[3,2,1])
arches_soybean = archs_soybean.sort_values(by=[3,2,1])

In [90]:
print(arches_corn.shape[0])
print(arches_wheat.shape[0])
print(arches_soybean.shape[0])

457
424
440


In [266]:
dates_corn = []
for ind in arches_corn.index:
    dates_corn.append(datetime(int(arches_corn[3].loc[ind]), int(arches_corn[2].loc[ind]), int(arches_corn[1].loc[ind])))

In [97]:
lst[arches_corn.index[0]]

'corn_11_10_2016_settlements.txt'

In [93]:
doc = []
with open('C:/Users/PC/Desktop/Settlements - Copy/' + lst[arches_corn.index[0]]) as f:
    for row in f:
        doc.append(row)

In [98]:
'STRIKE','OPEN','HIGH','LOW','LAST','SETT', 'CHGE' ,'EST.VOL','SETT', 'VOL', 'INT'

('STRIKE',
 'OPEN',
 'HIGH',
 'LOW',
 'LAST',
 'SETT',
 'CHGE',
 'EST.VOL',
 'SETT',
 'VOL',
 'INT')

In [108]:
doc[0:30]

['C Corn Futures\n',
 "DEC16     343'0     347'0     341'0     345'0     345'4    +2'2      137172      343'2      112526      706401\n",
 "MAR17     352'6     356'4     351'0     355'0     355'2    +2'0       46241      353'2       23868      268560\n",
 "MAY17     360'0     363'2     358'0     361'4     361'6    +1'4       12993      360'2        4744       68479\n",
 "JLY17     366'6     370'0     365'0     368'4     368'4    +1'4       10972      367'0        8898      123321\n",
 "SEP17     373'6     376'6     372'0     375'4     375'4    +1'4        1439      374'0         928       42938\n",
 "DEC17     382'4     385'6     381'0     382'0     384'2    +1'2        5018      383'0        3614       77091\n",
 "MAR18     391'0     394'6     390'6A    393'2A    393'4    +1'0         231      392'4          10        5394\n",
 "MAY18     397'2     397'4     397'2     397'4     398'4     +'4         166      398'0           3        1085\n",
 "JLY18     402'0     403'2B    401'0A    4

In [101]:
py = [d.startswith('PY') for d in doc] #corn contract name
py_index = pd.DataFrame(doc)[py].index

In [276]:
text = [re.sub("\s+", ",", d.strip()).split(',') for d in doc]
df = pd.DataFrame(text)
df = df[[0,1,2,3,4,8,10]]
df['expiration'] = np.zeros(df.shape[0])
df['tplus1'] = dates_corn[0]
#df.columns = ['strike', 'sett', 'oi']

In [277]:
df.head()

,0,1,2,3,4,8,10,expiration,tplus1
0,C,Corn,Futures,None,None,None,None,0.0,2016-09-21
1,DEC16,343'0,347'0,341'0,345'0,343'2,706401,0.0,2016-09-21
2,MAR17,352'6,356'4,351'0,355'0,353'2,268560,0.0,2016-09-21
3,MAY17,360'0,363'2,358'0,361'4,360'2,68479,0.0,2016-09-21
4,JLY17,366'6,370'0,365'0,368'4,367'0,123321,0.0,2016-09-21


In [278]:
for ind in df.index:
    num = str(df[[8]].iloc[ind]).split()[1].split('\'')
    try:
        if num[0] == '':
            num[0] = 0
        num = 10*float(num[0]) + float(num[1])
        df[8].loc[ind] = num
    except:
        pass
    

C:\Users\PC\Anaconda3\envs\cs109a\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [279]:
futs = df[[0,8]][1:6]
futs

,0,8
1,DEC16,3432
2,MAR17,3532
3,MAY17,3602
4,JLY17,3670
5,SEP17,3740


In [242]:
# Agricultural products expire on the 14th of every month, or the previous business day if a weekend/holiday

In [282]:
from pandas.tseries.offsets import BDay
mon = ['JAN','FEB',"MAR",'APR','MAY','JUN','JLY','AUG','SEP','OCT','NOV','DEC']
dummy_dict = {i : mon[i-1] for i in range(1,13)}
exp_dict = {}
for year in [16,17,18,19,20]:
    for month in range(1,13):
        exp_dict[mon[month - 1]+str(year)] = datetime(2000 + year, month, 15) - BDay(1)

exp_dict['JAN16']

Timestamp('2016-01-14 00:00:00')

In [303]:
df.iloc[py_index][4]
py_index_call = py_index[df.iloc[py_index][4] == 'CALL']
py_index_put = py_index[df.iloc[py_index][4] == 'PUT']

In [304]:
py_index_call

Int64Index([971, 1012, 1094, 1135, 1187, 1239, 1293, 1340, 1388, 1431, 1480], dtype='int64')

In [296]:
corn_vol_dict = {}
for i in range(7):
    last_ind = py_index[i]
    exp = str(df.iloc[py_index[i]][1])
    if exp in list(futs[0]):
        while df[0][last_ind + 1].isnumeric():
            last_ind += 1 
        df.expiration[py_index[i]:last_ind] = exp_dict[exp]
        corn_vol_dict[i] = df[py_index[i]:last_ind][1:].dropna()
        corn_vol_dict[i] = corn_vol_dict[i][[0,8,10,'expiration','tplus1']]
        corn_vol_dict[i] = corn_vol_dict[i].rename(columns={0:'strike',8:'settle',10:'oi'})
    else:
        corn_vol_dict[i] = ''

C:\Users\PC\Anaconda3\envs\cs109a\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [297]:
corn_vol_dict[3]

,strike,settle,oi,expiration,tplus1
1154,3500,193,21770,2017-03-14 00:00:00,2016-09-21
1155,3600,147,8387,2017-03-14 00:00:00,2016-09-21
1156,3700,112,9583,2017-03-14 00:00:00,2016-09-21
1157,3800,85,10673,2017-03-14 00:00:00,2016-09-21
1158,3900,64,9108,2017-03-14 00:00:00,2016-09-21
1159,4000,50,13155,2017-03-14 00:00:00,2016-09-21
1160,4100,37,4272,2017-03-14 00:00:00,2016-09-21
1161,4200,30,5096,2017-03-14 00:00:00,2016-09-21
1162,4300,24,2811,2017-03-14 00:00:00,2016-09-21
1164,4500,15,4555,2017-03-14 00:00:00,2016-09-21


In [100]:
py = [d.startswith('PY') for d in doc] #corn contract name
czo = [d.startswith('CZO') for d in doc] #soybeans contract name
wz = [d.startswith('WZ') for d in doc] #wheat contract name
ew_index = pd.DataFrame(doc)[ew].index
ez_index = pd.DataFrame(doc)[ez].index

ValueError: Item wrong length 18628 instead of 2083.

In [33]:
ew_index

Int64Index([2021, 2276, 2531, 2786, 3027, 3268, 3511, 3766, 4021, 4276, 4517,
            4758],
           dtype='int64')

In [29]:
text = [re.sub("\s+", ",", d.strip()).split(',') for d in doc]

In [53]:
df = pd.DataFrame(text)[[0,8]]
df.columns = ['strike', 'oi']

In [54]:
last_ind = ew_index[0] 
while df.strike[last_ind + 1].isnumeric():
    last_ind += 1

In [57]:
df[ew_index[0]:last_ind][1:].dropna()

,strike,oi
2049,1480,9
2052,1510,5
2056,1540,12
2057,1550,12
2060,1575,7
2069,1650,1
2074,1690,11
2077,1720,13
2085,1775,19
2100,1850,11


In [50]:
df = pd.DataFrame(text).iloc[ew_index[0]:ew_index[1]][[0,8]][1:]
df.columns = ['strike', 'oi']
df.dropna()

,strike,oi
2049,1480,9
2052,1510,5
2056,1540,12
2057,1550,12
2060,1575,7
2069,1650,1
2074,1690,11
2077,1720,13
2085,1775,19
2100,1850,11
